# Wrangle

In [73]:
#imports
import pandas as pd
import numpy as np

# system
import sys
# sys.path.append("../data")
sys.path.append("util_")
import wrangle_

In [61]:
# lacd data
df = pd.read_csv("data/known_exploited_vulnerabilities.csv")
df.head()

,cveID,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,knownRansomwareCampaignUse,notes,cwes
0,CVE-2021-27104,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
1,CVE-2021-27102,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
2,CVE-2021-27101,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
3,CVE-2021-27103,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,Known,NaN,NaN
4,CVE-2021-21017,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,Unknown,NaN,NaN


In [19]:
# look at the shape
df.shape

(1140, 11)

In [21]:
#look at the columns
df.columns

Index(['cveID', 'vendorProject', 'product', 'vulnerabilityName', 'dateAdded',
       'shortDescription', 'requiredAction', 'dueDate',
       'knownRansomwareCampaignUse', 'notes', 'cwes'],
      dtype='object')

In [63]:
# check the full info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cveID                       1140 non-null   object 
 1   vendorProject               1140 non-null   object 
 2   product                     1140 non-null   object 
 3   vulnerabilityName           1140 non-null   object 
 4   dateAdded                   1140 non-null   object 
 5   shortDescription            1140 non-null   object 
 6   requiredAction              1140 non-null   object 
 7   dueDate                     1140 non-null   object 
 8   knownRansomwareCampaignUse  1140 non-null   object 
 9   notes                       373 non-null    object 
 10  cwes                        0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 98.1+ KB


In [33]:
# select numeric
len(df.select_dtypes("number").columns) # count of numeric columns

1

In [55]:
#select object columns
len(df.select_dtypes("object").columns) #count of object columns

10

In [57]:
df.describe()

,cwes
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


**What I see:**


- I have 1140 rows and 11columns
- 1 of the 11 columns are numric while 10 of them are string object colums
- the numeric column `cwes` is null
- the `notes` column has 373 non null rows
- descriptive statistics says nothing on the `cwes` empty column